<a href="https://colab.research.google.com/github/FabianaAlbuquerque97/Busca-Inteligente-em-Grafos-Geograficos/blob/main/Busca_Inteligente_em_Grafos_Geogr%C3%A1ficos_%E2%80%93_Vers%C3%A3o_Interativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Bibliotecas
!pip install networkx gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00


In [2]:
import networkx as nx
import gradio as gr
import matplotlib.pyplot as plt
import json


In [6]:
with open('cities.json', 'r') as f:
    data = json.load(f)

graph = nx.Graph()
positions = {}

# Criar nós com posições
for city in data:
    name = city["city"]
    lat = city["latitude"]
    lon = city["longitude"]
    graph.add_node(name)
    positions[name] = (lon, lat)  # longitude como x, latitude como y

# Exemplo: criar arestas entre cidades próximas
# Aqui você pode definir a lógica para criar as conexões.
# Por exemplo, conectar cidades que estejam a até 500 km de distância

from geopy.distance import geodesic

threshold_km = 500

for i, city1 in enumerate(data):
    for j in range(i + 1, len(data)):
        city2 = data[j]
        coords_1 = (city1["latitude"], city1["longitude"])
        coords_2 = (city2["latitude"], city2["longitude"])
        distance = geodesic(coords_1, coords_2).km
        if distance <= threshold_km:
            graph.add_edge(city1["city"], city2["city"], weight=distance)


In [7]:
# 🔍 Algoritmos de busca
def dfs(graph, start, goal, path=None, visited=None):
    if path is None:
        path = [start]
    if visited is None:
        visited = set()
    visited.add(start)
    if start == goal:
        return path
    for neighbor in graph.neighbors(start):
        if neighbor not in visited:
            new_path = dfs(graph, neighbor, goal, path + [neighbor], visited)
            if new_path:
                return new_path
    return None

def greedy(graph, start, goal):
    from queue import PriorityQueue
    def heuristic(n1, n2):
        x1, y1 = positions[n1]
        x2, y2 = positions[n2]
        return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

    visited = set()
    pq = PriorityQueue()
    pq.put((0, [start]))
    while not pq.empty():
        _, path = pq.get()
        current = path[-1]
        if current == goal:
            return path
        if current not in visited:
            visited.add(current)
            for neighbor in graph.neighbors(current):
                if neighbor not in visited:
                    pq.put((heuristic(neighbor, goal), path + [neighbor]))
    return None


In [8]:
# 🖼️ Geração do gráfico com o caminho
def plot_path(path):
    plt.figure(figsize=(10, 8))
    nx.draw(graph, positions, node_color='lightblue', with_labels=True, node_size=500)
    if path:
        edge_path = list(zip(path, path[1:]))
        nx.draw_networkx_edges(graph, positions, edgelist=edge_path, edge_color='r', width=2)
    plt.title("Caminho encontrado")
    plt.axis('off')
    plt.savefig("path_plot.png")
    return "path_plot.png"


In [9]:
# 🎯 Função principal para rodar via Gradio
def run_search(start, goal, method):
    if method == "DFS":
        path = dfs(graph, start, goal)
    elif method == "Greedy":
        path = greedy(graph, start, goal)
    else:
        return "Método inválido", None
    if path:
        img = plot_path(path)
        return f"Caminho encontrado: {' → '.join(path)}", img
    else:
        return "Caminho não encontrado.", None


In [10]:
# 🧭 Interface Interativa
cities = list(graph.nodes)

interface = gr.Interface(
    fn=run_search,
    inputs=[
        gr.Dropdown(choices=cities, label="Cidade de Origem"),
        gr.Dropdown(choices=cities, label="Cidade de Destino"),
        gr.Radio(choices=["DFS", "Greedy"], label="Algoritmo de Busca")
    ],
    outputs=[
        gr.Text(label="Resultado"),
        gr.Image(type="filepath", label="Visualização do Caminho")
    ],
    title="Busca Inteligente em Grafos Geográficos",
    description="Escolha a origem, destino e algoritmo de busca para simular rotas em um grafo geográfico real."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3bb9be4ac7866b791.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
